# ETL_WineXL_CleanUP

#### Comments on ETL_WineXL_CleanUp
###### 1) Since the file we were cleaning up was a .xlxs, we needed to install openpyxl and then use that as our engine to read the excel file
###### 2) Eliminated Vintage, County and Designation columns 
###### 3) Determined that there were incomplete rows and then eliminated those rows
###### 4) Evaluated data to determine if the data was of the appropriate type. Noted that Price was an object rather than a float
###### 5) Converted the price column to a float
###### 6) Eliminated all wines that were not include in the top 6 favorite wines per the Art Wine Preserve website
###### 7) Reindexed the DataFrame after eliminating non-top 6 wines

In [15]:
import pandas as pd
from sqlalchemy import create_engine
#!pip install openpyxl
#!pip install psycopg2
import psycopg2 as pg2

In [ ]:
#### Store xlsx into a DataFrame

In [16]:
data_file = "Resources/Wines.xlsx"
df = pd.read_excel(data_file, engine='openpyxl')
df.head()

,Vintage,Country,County,Designation,Points,Price,Province,Title,Variety,Winery
0,1919-01-01 00:00:00,Spain,Cava,1919 Brut Selecció,88,$13.00,Catalonia,L'Arboc NV 1919 Brut Selecció Sparkling (Cava),Sparkling Blend,L'Arboc
1,1929-01-01 00:00:00,Italy,Vernaccia di San Gimignano,NaN,87,$14.00,Tuscany,Guidi 1929 2015 Vernaccia di San Gimignano,Vernaccia,Guidi 1929
2,1929-01-01 00:00:00,Italy,Sangiovese di Romagna Superiore,Prugneto,84,$15.00,Central Italy,Poderi dal Nespoli 1929 2011 Prugneto (Sangiov...,Sangiovese,Poderi dal Nespoli 1929
3,1934-01-01 00:00:00,Portugal,NaN,Reserva Velho,93,$495.00,Colares,Adega Viuva Gomes 1934 Reserva Velho Red (Cola...,Ramisco,Adega Viuva Gomes
4,1945-01-01 00:00:00,France,Rivesaltes,Legend Vintage,95,$350.00,Languedoc-Roussillon,Gérard Bertrand 1945 Legend Vintage Red (Rives...,Red Blend,Gérard Bertrand


In [17]:
pd.Series(type(x) for x in df['Vintage']).unique()



array([<class 'datetime.datetime'>, <class 'str'>], dtype=object)

In [18]:
df["Vintage"] = pd.to_datetime(df['Vintage'], infer_datetime_format=True)
df.head()



,Vintage,Country,County,Designation,Points,Price,Province,Title,Variety,Winery
0,1919-01-01,Spain,Cava,1919 Brut Selecció,88,$13.00,Catalonia,L'Arboc NV 1919 Brut Selecció Sparkling (Cava),Sparkling Blend,L'Arboc
1,1929-01-01,Italy,Vernaccia di San Gimignano,NaN,87,$14.00,Tuscany,Guidi 1929 2015 Vernaccia di San Gimignano,Vernaccia,Guidi 1929
2,1929-01-01,Italy,Sangiovese di Romagna Superiore,Prugneto,84,$15.00,Central Italy,Poderi dal Nespoli 1929 2011 Prugneto (Sangiov...,Sangiovese,Poderi dal Nespoli 1929
3,1934-01-01,Portugal,NaN,Reserva Velho,93,$495.00,Colares,Adega Viuva Gomes 1934 Reserva Velho Red (Cola...,Ramisco,Adega Viuva Gomes
4,1945-01-01,France,Rivesaltes,Legend Vintage,95,$350.00,Languedoc-Roussillon,Gérard Bertrand 1945 Legend Vintage Red (Rives...,Red Blend,Gérard Bertrand


In [19]:
df["Vintage"] = pd.Series(x.year for x in df["Vintage"])
df.head()



,Vintage,Country,County,Designation,Points,Price,Province,Title,Variety,Winery
0,1919,Spain,Cava,1919 Brut Selecció,88,$13.00,Catalonia,L'Arboc NV 1919 Brut Selecció Sparkling (Cava),Sparkling Blend,L'Arboc
1,1929,Italy,Vernaccia di San Gimignano,NaN,87,$14.00,Tuscany,Guidi 1929 2015 Vernaccia di San Gimignano,Vernaccia,Guidi 1929
2,1929,Italy,Sangiovese di Romagna Superiore,Prugneto,84,$15.00,Central Italy,Poderi dal Nespoli 1929 2011 Prugneto (Sangiov...,Sangiovese,Poderi dal Nespoli 1929
3,1934,Portugal,NaN,Reserva Velho,93,$495.00,Colares,Adega Viuva Gomes 1934 Reserva Velho Red (Cola...,Ramisco,Adega Viuva Gomes
4,1945,France,Rivesaltes,Legend Vintage,95,$350.00,Languedoc-Roussillon,Gérard Bertrand 1945 Legend Vintage Red (Rives...,Red Blend,Gérard Bertrand


In [20]:
df.dtypes

Vintage         int64
Country        object
County         object
Designation    object
Points          int64
Price          object
Province       object
Title          object
Variety        object
Winery         object
dtype: object

#### Eliminate unnecessary columns

In [21]:

new_df = df[['Vintage','Country', 'Points', "Price", 'Province', 'Title', 'Variety', 'Winery']].copy()
new_df.head()

,Vintage,Country,Points,Price,Province,Title,Variety,Winery
0,1919,Spain,88,$13.00,Catalonia,L'Arboc NV 1919 Brut Selecció Sparkling (Cava),Sparkling Blend,L'Arboc
1,1929,Italy,87,$14.00,Tuscany,Guidi 1929 2015 Vernaccia di San Gimignano,Vernaccia,Guidi 1929
2,1929,Italy,84,$15.00,Central Italy,Poderi dal Nespoli 1929 2011 Prugneto (Sangiov...,Sangiovese,Poderi dal Nespoli 1929
3,1934,Portugal,93,$495.00,Colares,Adega Viuva Gomes 1934 Reserva Velho Red (Cola...,Ramisco,Adega Viuva Gomes
4,1945,France,95,$350.00,Languedoc-Roussillon,Gérard Bertrand 1945 Legend Vintage Red (Rives...,Red Blend,Gérard Bertrand


#### Check to see if all rows have complete data

In [22]:
new_df.count()

Vintage     24997
Country     24989
Points      24997
Price       23375
Province    24989
Title       24997
Variety     24997
Winery      24997
dtype: int64

#### Eliminate rows with incomplete data

In [23]:
new_df = new_df.dropna(how='any')
new_df.count()

Vintage     23367
Country     23367
Points      23367
Price       23367
Province    23367
Title       23367
Variety     23367
Winery      23367
dtype: int64

#### Determine if all columns are set up in the correct data type

In [24]:
new_df.dtypes

Vintage      int64
Country     object
Points       int64
Price       object
Province    object
Title       object
Variety     object
Winery      object
dtype: object

###### Note: price should be a float

#### Convert the Price columnt to float

In [25]:
new_df['Price'] = [float(x.replace("$","").replace(",","")) for x in new_df['Price']]
new_df.dtypes

Vintage       int64
Country      object
Points        int64
Price       float64
Province     object
Title        object
Variety      object
Winery       object
dtype: object

In [26]:
new_df = new_df.loc[(new_df['Country'] == "US")]
new_df
                       

,Vintage,Country,Points,Price,Province,Title,Variety,Winery
11,1987,US,89,170.0,California,Sebastiani 1987 Cherryblock Cabernet Sauvignon...,Cabernet Sauvignon,Sebastiani
15,1991,US,93,60.0,California,Iron Horse 1991 Brut L.D. (Sonoma County),Champagne Blend,Iron Horse
16,1992,US,88,22.0,California,Gloria Ferrer 1992 Royal Cuvee Brut (Carneros),Champagne Blend,Gloria Ferrer
17,1992,US,91,32.0,California,Pacific Echo 1992 Private Reserve Brut (Anders...,Champagne Blend,Pacific Echo
19,1993,US,86,24.0,Oregon,Rex Hill 1993 Oregon,Champagne Blend,Rex Hill
...,...,...,...,...,...,...,...,...
24977,2016,US,91,23.0,California,Saracina 2016 Sauvignon Blanc (Mendocino County),Sauvignon Blanc,Saracina
24978,2016,US,86,16.0,New York,Hosmer 2016 Estate Winery Pinot Gris (Cayuga L...,Pinot Gris,Hosmer
24980,2016,US,90,22.0,California,C R Graybehl 2016 Grenache Rosé (Sonoma Valley),Rosé,C R Graybehl
24984,2016,US,90,23.0,California,Belharra 2016 Las Madres Vineyards Rosé of Syr...,Syrah,Belharra


#### Isolate the top 6 wine types per https://artwinepreserver.com/: Cabernet Sauvignon, Chardonnay, Pinot Gris/Pinot Grigio, Pinot Noir, Sauvignon Blanc, Merlot (Pinot Gris/Pinot Grigio - Pinot Gris in this dataset)

In [27]:
winexl_df = new_df.loc[(new_df['Variety'] == "Cabernet Sauvignon") | (new_df['Variety'] == "Chardonnay") 
                       | (new_df['Variety'] == "Pinot Gris") | (new_df['Variety'] == "Pinot Noir")
                       | (new_df['Variety'] == "Sauvignon Blanc") | (new_df['Variety'] == "Merlot")]
count = winexl_df['Variety'].value_counts()
count

Pinot Noir            1953
Cabernet Sauvignon    1428
Chardonnay            1336
Sauvignon Blanc        459
Merlot                 419
Pinot Gris             172
Name: Variety, dtype: int64

In [28]:
winexl_df.head()

,Vintage,Country,Points,Price,Province,Title,Variety,Winery
11,1987,US,89,170.0,California,Sebastiani 1987 Cherryblock Cabernet Sauvignon...,Cabernet Sauvignon,Sebastiani
23,1994,US,82,13.0,California,Gan Eden 1994 Chardonnay (Sonoma County),Chardonnay,Gan Eden
36,1996,US,83,22.0,California,Meridian 1996 Coastal Reserve Cabernet Sauvign...,Cabernet Sauvignon,Meridian
37,1996,US,84,29.0,Washington,Covey Run 1996 Whiskey Canyon Vyd Cabernet Sau...,Cabernet Sauvignon,Covey Run
38,1996,US,88,35.0,California,Château Souverain 1996 Winemaker's Reserve Cab...,Cabernet Sauvignon,Château Souverain


In [29]:
winexl_df = winexl_df.reset_index(drop=True)
winexl_df.head()

,Vintage,Country,Points,Price,Province,Title,Variety,Winery
0,1987,US,89,170.0,California,Sebastiani 1987 Cherryblock Cabernet Sauvignon...,Cabernet Sauvignon,Sebastiani
1,1994,US,82,13.0,California,Gan Eden 1994 Chardonnay (Sonoma County),Chardonnay,Gan Eden
2,1996,US,83,22.0,California,Meridian 1996 Coastal Reserve Cabernet Sauvign...,Cabernet Sauvignon,Meridian
3,1996,US,84,29.0,Washington,Covey Run 1996 Whiskey Canyon Vyd Cabernet Sau...,Cabernet Sauvignon,Covey Run
4,1996,US,88,35.0,California,Château Souverain 1996 Winemaker's Reserve Cab...,Cabernet Sauvignon,Château Souverain


In [30]:
#Create connection with PostgreSQL
#rds_connection_string = "<postgress>:<Todayis11222020!>@localhost:5432/customer_db"
#conn = pg2.connect(database='wines',user='postgres', password=Todayis11222020!)
#engine = create_engine(f'postgresql://{rds_connection_string}')